<a href="https://colab.research.google.com/github/WakWakBird/MLS/blob/main/tf2-10-4-mnist_nn_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


## Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [ ]:
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)

# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.862475891000031
GPU (s):
0.10837535100017703
GPU speedup over CPU: 35x


# Lab


In [16]:
# Lab 10: MNIST Deep Neural Network with Glorot Initialization
import numpy as np
import random
import tensorflow as tf

# 재현성 설정
random.seed(777)
learning_rate = 0.001
batch_size = 100
training_epochs = 15
nb_classes = 10  # 0~9 숫자

# 1. 데이터 로딩
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print("x_train.shape:", x_train.shape)  # (60000, 28, 28)

# 2. 전처리 (reshape + 정규화)
x_train = x_train.reshape(-1, 28 * 28) / 255.0
x_test = x_test.reshape(-1, 28 * 28) / 255.0

# 3. One-hot 인코딩
y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

# 4. 모델 구성 (은닉층 4개 + 출력층)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=512, input_dim=784,
                                 kernel_initializer='glorot_normal', activation='relu'))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer='glorot_normal', activation='relu'))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer='glorot_normal', activation='relu'))
model.add(tf.keras.layers.Dense(units=512, kernel_initializer='glorot_normal', activation='relu'))
model.add(tf.keras.layers.Dense(units=nb_classes, kernel_initializer='glorot_normal', activation='softmax'))

# 5. 컴파일 (✅ lr → learning_rate로 수정)
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    metrics=['accuracy']
)

# 6. 모델 구조 출력
model.summary()

# 7. 학습
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=training_epochs)

# 8. 예측 결과 샘플 출력
y_predicted = model.predict(x_test)
for i in range(10):
    rand_index = random.randint(0, x_test.shape[0] - 1)
    actual = np.argmax(y_test[rand_index])
    predicted = np.argmax(y_predicted[rand_index])
    print(f"index: {rand_index}, actual y: {actual}, predicted y: {predicted}")

# 9. 전체 테스트셋 평가
evaluation = model.evaluate(x_test, y_test)
print("\n🧾 Test Loss:", evaluation[0])
print("✅ Test Accuracy:", evaluation[1])


x_train.shape: (60000, 28, 28)


Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_29 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_32 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,195,018 (4.56 MB)

 Trainable params: 1,195,018 (4.56 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 16s 25ms/step - accuracy: 0.8821 - loss: 0.3795
Epoch 2/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.9716 - loss: 0.0903
Epoch 3/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 21s 25ms/step - accuracy: 0.9807 - loss: 0.0614
Epoch 4/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.9859 - loss: 0.0432
Epoch 5/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.9886 - loss: 0.0359
Epoch 6/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.9900 - loss: 0.0320
Epoch 7/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 22s 27ms/step - accuracy: 0.9902 - loss: 0.0320
Epoch 8/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.9936 - loss: 0.0221
Epoch 9/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 20s 25ms/step - accuracy: 0.9931 - loss: 0.0236
Epoch 10/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.9948 - loss: 0.0160
Epoch 11/15
600/600 ━━━━━━━━━━━━━━━━━━━━ 16s 26ms/step - accuracy: 0.9951 - loss: 0.0165
Epoch 12/15
600/600 ━━━━━━━━━━